In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load the image
image = cv2.imread('input_image.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding to extract the object
_, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)

# Find contours of the object
contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a blank canvas
canvas = np.zeros_like(image)

# Loop over the contours
for contour in contours:
    # Calculate the bounding box of the contour
    x, y, w, h = cv2.boundingRect(contour)

    # Add padding to the bounding box
    padding = 20
    x -= padding
    y -= padding
    w += 2 * padding
    h += 2 * padding

    # Crop the region containing the tires
    tire_region = image[y:y+h, x:x+w]

    # Create a mask for the tire region
    mask = np.zeros_like(tire_region)
    cv2.drawContours(mask, [contour - (x, y)], 0, (255, 255, 255), -1)

    # Apply a Gaussian blur to the mask
    blurred_mask = cv2.GaussianBlur(mask, (25, 25), 0)

    # Offset the blurred mask downwards to create the shadow effect
    shadow_offset = 20
    M = np.float32([[1, 0, 0], [0, 1, shadow_offset]])
    shifted_mask = cv2.warpAffine(blurred_mask, M, (w, h))

    # Combine the original tire region and the shifted mask
    shadow = cv2.addWeighted(tire_region, 1, shifted_mask, 0.7, 0)

    # Replace the tire region with the shadow
    canvas[y:y+h, x:x+w] = shadow

# Display the result using matplotlib
plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
